In [ ]:
import requests
import json
from pprint import pprint
from config import zillow_key
import pandas as pd


In [ ]:
base_url = f'https://data.nasdaq.com/api/v3/datatables/ZILLOW/REGIONS?api_key={zillow_key}'
response = requests.get(base_url)
data = response.json()
pprint(data)
filter_string = "Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County"
filtered_data = [
    entry for entry in data['datatable']['data']
    if filter_string in entry[2]
]
pprint(filtered_data)


In [ ]:
regions_columns = ['region_id', 'region_type', 'region']
region_zillow_df = pd.DataFrame(filtered_data, columns=regions_columns)
region_zillow_df.to_csv('zillow_region.csv', index=False)

In [ ]:
extracted_ids = [entry[0] for entry in filtered_data]
zip_ids = [[entry[0], entry[2]] for entry in filtered_data]
pprint(zip_ids)


In [ ]:
location_map = {entry[0]: entry[1] for entry in zip_ids}
indicators = ["ZSFH", "ZCON", "Z5BR"]
region_ids = extracted_ids
second_url = "https://data.nasdaq.com/api/v3/datatables/ZILLOW/DATA"
all_data = []
for region_id in region_ids:
    for indicator in indicators:
        url = f"{second_url}?indicator_id={indicator}&region_id={region_id}&api_key={zillow_key}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for item in data['datatable']['data']:
                full_location = location_map.get(region_id, "Location not found")
                zip_code = full_location.split(';')[0]
                item.append(zip_code)
                all_data.append(item)
        else:
            print(f"Failed to retrieve data for region_id {region_id}, indicator {indicator}")
pprint(all_data)


In [ ]:
columns = ['indicator_id', 'region_id', 'date', 'value', 'zip']
zillow_df = pd.DataFrame(all_data, columns=columns)
zillow_df['date'] = pd.to_datetime(zillow_df['date'])
zillow_df
zillow_df.to_csv('zillow_data.csv', index=False)

In [ ]:
third_url = f'https://data.nasdaq.com/api/v3/datatables/ZILLOW/INDICATORS?api_key={zillow_key}'
response_indicators = requests.get(third_url)
data_indicators = response_indicators.json()
desired_indicators = ["ZSFH", "ZCON", "Z5BR"]

filtered_indicators = [
    entry for entry in data_indicators['datatable']['data']
    if entry[0] in desired_indicators
]

pprint(filtered_indicators)


In [ ]:
indicators_columns = ['indicators_id', 'indicators', 'category']
indicators_zillow_df = pd.DataFrame(filtered_indicators, columns=indicators_columns)
indicators_zillow_df.to_csv('zillow_indicators.csv', index=False)